In [ ]:
import os
import pickle
import shutil
from pathlib import Path
from synplan.utils.loading import download_all_data

# download SynPlanner data
data_folder = Path("synplan_data").resolve()
download_all_data(save_to=data_folder)

# input data
# You can use the input data:
# 1. Curated USPTO data, extracted reaction rules, pretrained policy network and building blocks from downloaded data
# 2. Use curated USPTO data, extracted reaction rules, pretrained policy network prepared with tutorials

USE_DOWNLOAD_DATA = True # is False if you want to use the data and models from previous tutorial steps
if USE_DOWNLOAD_DATA:
    # input data
    ranking_policy_network = data_folder.joinpath("uspto/weights/ranking_policy_network.ckpt").resolve(strict=True)
    reaction_rules_path = data_folder.joinpath("uspto/uspto_reaction_rules.pickle")
    # planning reslts folder
    results_folder = Path("planning_with_downloaded_data").resolve()
    results_folder.mkdir(exist_ok=True)
else:
    # input data
    tutorial_results_folder = Path("tutorial_results").resolve()
    ranking_policy_network = tutorial_results_folder.joinpath("ranking_policy_network/policy_network.ckpt").resolve(strict=True)
    reaction_rules_path = tutorial_results_folder.joinpath("uspto_reaction_rules.pickle")
    # planning reslts folder
    results_folder = Path("planning_with_tutorial_data").resolve()
    results_folder.mkdir(exist_ok=True)

# use your custom building blocks if needed
building_blocks_path = data_folder.joinpath("building_blocks/building_blocks_em_sa_ln.smi")

In [ ]:
from synplan.utils.loading import load_building_blocks

building_blocks = load_building_blocks(building_blocks_path, standardize=True)

In [3]:
from synplan.utils.loading import load_reaction_rules

reaction_rules = load_reaction_rules(reaction_rules_path)

In [ ]:
from synplan.mcts.expansion import PolicyNetworkFunction
from synplan.utils.config import PolicyNetworkConfig

policy_config = PolicyNetworkConfig(weights_path=ranking_policy_network) 
policy_network = PolicyNetworkFunction(policy_config=policy_config)

In [5]:
from synplan.utils.config import TreeConfig

tree_config = TreeConfig(
    search_strategy="expansion_first",
    evaluation_type="rollout",
    max_iterations=300,
    max_time=120,
    max_depth=9,
    min_mol_size=1,
    init_node_value=0.5,
    ucb_type="uct",
    c_ucb=0.1,
)

In [6]:
from synplan.chem.utils import mol_from_smiles

# let's take capivasertib used as anti-cancer medication for the treatment 
# of breast cancer and approved by FDA in 2023
example_smiles = "NC1(C(=O)N[C@@H](CCO)c2ccc(Cl)cc2)CCN(c2nc[nH]c3nccc2-3)CC1"

target_molecule = mol_from_smiles(example_smiles, clean2d=True, standardize=True, clean_stereo=True)

In [7]:
from synplan.mcts.tree import Tree

tree = Tree(
    target=target_molecule,
    config=tree_config,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    expansion_function=policy_network,
    # you can also specify evaluation_function=ValueNetwork(...), by default it is None
)

In [ ]:
tree_solved = False
for solved, node_id in tree:
    if solved:
        tree_solved = True
tree

### Extract RouteCGR and ReducedRouteCGR

In [9]:
from synplan.chem.reaction_routes.route_cgr import *
from synplan.chem.reaction_routes.clustering import *

In [151]:
all_route_cgrs = compose_all_route_cgrs(tree)

In [152]:
all_reduced_route_cgrs = compose_all_reduced_route_cgrs(all_route_cgrs)

### Clustering

In [153]:
clusters = cluster_routes(all_reduced_route_cgrs, use_strat= False)

In [ ]:
from synplan.utils.visualisation import routes_clustering_report, routes_subclustering_report
from IPython.display import display, HTML

cluster_index = '3.2'
if cluster_index in clusters.keys():
    display(HTML(routes_clustering_report(tree, clusters, cluster_index,
                         all_reduced_route_cgrs)))

### Subclustering

In [155]:
all_subclusters = subcluster_all_clusters(clusters, all_reduced_route_cgrs, all_route_cgrs)

In [ ]:
subcluster_num = 1

if subcluster_num in all_subclusters[cluster_index].keys():
    subgroup = all_subclusters[cluster_index][subcluster_num]
    display(HTML(routes_subclustering_report(tree, subgroup, cluster_index, subcluster_num, all_reduced_route_cgrs, aam=False)))

In [ ]:


if len(subgroup['nodes_data']) != 1:
    new_subgroup = post_process_subgroup(subgroup)
    display(HTML(routes_subclustering_report(tree, new_subgroup, cluster_index, subcluster_num, all_reduced_route_cgrs, if_lg_group=True)))